In [1]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
llm.invoke("HELLO")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-47868991-b396-4c58-b52d-495237047b04-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain.agents import tool 

@tool 
def get_word_length(word: str) -> int: 
    """Returns the length of a word""" 
    return len(word) 

get_word_length.invoke("abc")

3

In [3]:
tools = [get_word_length] 

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [5]:
llm_with_tools = llm.bind_tools(tools) 

In [6]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages 
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser 

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x['intermediate_steps'])
    } | prompt | llm_with_tools | OpenAIToolsAgentOutputParser()
)

In [7]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 

In [8]:
list(agent_executor.stream({"input": "How many letters in the word hello"}))



> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'hello'}`


5The word "hello" has 5 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'hello'}, log="\nInvoking: `get_word_length` with `{'word': 'hello'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Vxl8cYtJR97ulQ3oCQDAliMF', 'function': {'arguments': '{"word":"hello"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0'}, id='run-cea9c6ca-40b1-44cd-91ec-54df6f5e2395', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'hello'}, 'id': 'call_Vxl8cYtJR97ulQ3oCQDAliMF', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"hello"}', 'id': 'call_Vxl8cYtJR97ulQ3oCQDAliMF', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Vxl8cYtJR97ulQ3oCQDAliMF')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Vxl8cYtJR97ulQ

In [9]:
MEMORY_KEY = "chat_history" 
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = [] 

In [12]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ), 
        "chat_history": lambda x: x["chat_history"]
    } | prompt | llm_with_tools | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
input1 = "How many letters in the word educa?" 
result = agent_executor.invoke({"input": input1, "chat_history": chat_history}) 
chat_history.extend([
    HumanMessage(content=input1), 
    AIMessage(content=result['output'])
])
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "educa" is a real word, though it is not commonly used in English. It is derived from the Latin word "educare," which means "to educate" or "to bring up." In some contexts, it may be used as a root in educational terminology. However, it is not a standard English word you would find in most dictionaries.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='How many letters in the word educa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "educa" has 5 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, "educa" is a real word, though it is not commonly used in English. It is derived from the Latin word "educare," which means "to educate" or "to bring up." In some contexts, it may be used as a root in educational terminology. However, it is not a standard English word you would find in most dictionaries.'}

In [14]:

agent_executor.invoke({"input": "How much is 1 Euro in USD?", "chat_history": chat_history})



> Entering new AgentExecutor chain...
I don't have real-time data access to provide the current exchange rate for Euro to USD. You may want to check a financial news website or a currency converter for the most accurate and up-to-date information.

> Finished chain.


{'input': 'How much is 1 Euro in USD?',
 'chat_history': [HumanMessage(content='How many letters in the word educa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "educa" has 5 letters.', additional_kwargs={}, response_metadata={})],
 'output': "I don't have real-time data access to provide the current exchange rate for Euro to USD. You may want to check a financial news website or a currency converter for the most accurate and up-to-date information."}

In [33]:
import requests
@tool
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()['rates'][currency_to]

tools = [get_exchange_rate] 


In [37]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 
llm_with_tools = llm.bind_tools(tools) 

In [38]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ), 
        "chat_history": lambda x: x["chat_history"]
    } | prompt | llm_with_tools | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [40]:
agent_executor.invoke({"input": "How much is 1 Euro in USD?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_exchange_rate` with `{'currency_from': 'EUR', 'currency_to': 'USD'}`


1.05621 Euro is approximately 1.0562 USD.

> Finished chain.


{'input': 'How much is 1 Euro in USD?',
 'chat_history': [HumanMessage(content='How many letters in the word educa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "educa" has 5 letters.', additional_kwargs={}, response_metadata={})],
 'output': '1 Euro is approximately 1.0562 USD.'}